# Projective Preferential Bayesian Optimization: Campor/Cu(111)

<font size="4">In this notebook the user's belief about an optimal configuration can be elicited by using the [Projective Preferential Bayesian Optimization](https://arxiv.org/abs/2002.03113) framework. The problem case is the adsorption of a non-symmetric bulky molecule camphor on the flat surface of (111)-plane terminated Cu slab.</font> 

#### Import dependencies

In [1]:
import time
import numpy as np
import Camphor_Copper.GUI as GUI
from gp_model import GPModel
from ppbo_settings import PPBO_settings
from acquisition import next_query
from IPython.display import display, HTML
import sys
from datetime import datetime

#### Define the problem setting and the aquisition strategy
There are six possible acquisition startegies:
- PCD = preferential coordinate descent
- EI = expected improvement by projective preferential query
- EXT = pure exploitation
- EXR = pure exploration (variance maximization)
- RAND = random 

In [2]:
acquisition_strategy = 'EI'

In [3]:
PPBO_settings = PPBO_settings(D=6,bounds=((-0.5,0.5),(-0.5,0.5),(4,7),(-180,180),(-180,180),(-180,180)),
                    xi_acquisition_function=acquisition_strategy,verbose=True)

#### Set initial queries

In [4]:
initial_queries_xi = np.array([list(np.eye(6)[i]) for i in range(6)]) #Initial xi:s correspond to unit vectors
initial_queries_x = np.array([[-0.5, -0.5, 5.0, -84.4, 142.8, 2.7],
                              [0.25, -0.25, 5.0, -84.4, 142.8, 2.7],
                              [-0.125, -0.125, 5.0, -84.4, 142.8, 2.7],
                              [-0.3147064250413807, -0.1379205809600735, 5.0, -84.4, 142.8, 2.7],
                              [-0.1420906798614234, -0.3133597318361268, 5.0, -84.4, 142.8, 2.7],
                              [0.3564088304603891, 0.3885800560423534, 5.0, -84.4, 142.8, 2.7]]) 
print("Number of initial queries is: " + str(len(initial_queries_xi)))

Number of initial queries is: 6


#### Querying settings

In [5]:
NUMBER_OF_QUERIES = 12
ADAPTIVE_INITIALIZATION = True  #At initilization: immediatly update the coordinate according to the user feedback

#### Hyperparameter optimization

In [6]:
OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_ACTUAL_QUERY_NUMBER = 1000 

#### Initialize the user session

In [7]:
should_log = False
if should_log:
    orig_stdout = sys.stdout
    log_file = open('Camphor_Copper/user_session_log_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.txt', "w")
    sys.stdout = log_file
GUI_ses = GUI.GUI_session(PPBO_settings)
results_mu_star = []
results_x_star = []

## PPBO loops

In [8]:
start = time.time() 

### 1. Initialization loop

In [ ]:
for i in range(len(initial_queries_xi)):
    ''' Present query to the user '''
    xi = initial_queries_xi[i].copy()
    if not i==0 and GUI_ses.user_feedback is not None and ADAPTIVE_INITIALIZATION:
        initial_queries_x[i:,:] = GUI_ses.user_feedback
    x = initial_queries_x[i].copy()
    x[xi!=0] = 0
    GUI_ses.set_x(x)
    GUI_ses.set_xi(xi)
    GUI_ses.run_iteration(allow_feedback=True)
    ''' Create GP model for first time '''
    if i==0:
        GP_model = GPModel(PPBO_settings)
    ''' Update GP model '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.update_data()
    GP_model.update_model()
    ''' Save the predictive mean maximum and maximizer'''
    print("x_star of the iteration: " + str(GP_model.x_star_))
    results_mu_star.append(GP_model.mu_star_)
    results_x_star.append(GP_model.x_star_)   
if OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION:
    GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION)  
print("Initialization done!")

--- Feedback ---
Typed value: 50.0
... converted to: [ 9.50401108e-03 -5.00000000e-01  5.00000000e+00 -8.44000000e+01
  1.42800000e+02  2.70000000e+00]
Iteration done!
MAP-estimation begins...
Optimization terminated successfully.
         Current function value: 0.481588
         Iterations: 3
         Function evaluations: 4
         Gradient evaluations: 4
         Hessian evaluations: 4
... this took 0.027261734008789062 seconds.
Current theta is: [0.001, 20, 0.001] (Acq. = EI)
Updating Lambda_MAP...
... this took 0.0022644996643066406 seconds.
Updating posterior covariance...
... this took 0.0005552768707275391 seconds.
Computing mu_star and x_star ...
... this took 0.26085424423217773 seconds.
x_star of the iteration: [0.54852694 0.0034054  0.34408487 0.26422064 0.89621788 0.51712584]
--- Feedback ---
Typed value: 68.0
... converted to: [ 9.50401108e-03  1.70619314e-01  5.00000000e+00 -8.44000000e+01
  1.42800000e+02  2.70000000e+00]
Iteration done!
MAP-estimation begins...
     

### 2. Main loop

In [ ]:
for i in range(NUMBER_OF_QUERIES):
    print("Starting query " + str(i+1)+"/"+str(NUMBER_OF_QUERIES)+" ...")
    ''' Compute next query '''
    xi_next,x_next = next_query(PPBO_settings,GP_model,unscale=True)
    ''' Present this to the user '''
    GUI_ses.set_xi(xi_next)
    GUI_ses.set_x(x_next)
    GUI_ses.run_iteration(allow_feedback=True)
    ''' Append the user feedback '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.mu_star_previous_iteration = GP_model.mu_star_
    ''' Update the model '''
    GP_model.update_data()
    if i+1==OPTIMIZE_HYPERPARAMETERS_AFTER_ACTUAL_QUERY_NUMBER:
        GP_model.update_model(optimize_theta=True)     
    else:
        GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION)
    ''' Save the predictive mean maximum and maximizer'''
    print("x_star of the iteration: " + str(GP_model.x_star_))
    results_mu_star.append(GP_model.mu_star_)
    results_x_star.append(GP_model.x_star_)

In [ ]:
print("---------------------------------------")
print("The session completed!")
print("Total time: " + str(time.time()-start) + " seconds.")
x_star_, mu_star_  = GP_model.mu_star(mu_star_finding_trials=20)
x_star_unscaled = GP_model.FP.unscale(x_star_)
print("The final x_star: " + str(x_star_))

## Save the results

#### Generate html-file corresponding to the user's most preferred configuration

In [ ]:
optimal_configuration_html = GUI.generate_optimal_configuration(x_star_unscaled)

#### Save the user session results

In [ ]:
#Save results to csv-file
print("Saving the user session results...")
user_session_results = GUI_ses.results.copy()
user_session_results['iter_mu_star'] = results_mu_star
user_session_results['iter_x_star'] = results_x_star
user_session_results.to_csv('Camphor_Copper/user_session_results_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.csv',index=False)
#Close the log-file
if should_log:
    sys.stdout = orig_stdout
    log_file.close()

## Analyze the results

#### View the user's most preferred configuration
This open an ASE-package viewer. Press the button "i" to restore a default view.

In [ ]:
#HTML(filename=optimal_configuration_html)

#### Slice plots of the utility function (predictive mean)

In [ ]:
import Camphor_Copper.plot_results as pr
%matplotlib notebook

In [ ]:
pr.sliceplot_pred_mean('alpha','beta',GP_model,x_star_)

In [ ]:
pr.sliceplot_pred_mean('x','y',GP_model,x_star_)

In [ ]:
pr.sliceplot_pred_mean('z','gamma',GP_model,x_star_)

In [ ]:
print("The most preferred configuration (unscaled): " + str(list(GP_model.FP.unscale(x_star_))))

In [ ]:
print("The most preferred configuration (scaled): " + str(list(x_star_)))